In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
import networkx as nx
import matplotlib.pyplot as plt
import geopandas as gpd
import time
import geopandas
import geodatasets
import gdal
import pygame
from shapely.geometry import MultiPolygon, Polygon, Point

pygame 2.5.2 (SDL 2.28.3, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


C:\Users\thebu\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
gdf = gpd.read_file('taxi_zones.shp')

In [3]:
yellow_taxi_df =pd.read_parquet('yellow_tripdata_2023-01.parquet', engine='fastparquet')
yellow_taxi_df = yellow_taxi_df.head(150000)
zone_lookup_df = pd.read_csv("taxi+_zone_lookup.csv")

In [4]:
zone_lookup_df.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [5]:
class Graph:
    def __init__(self):
        self.adjacency_list = {}

    def add_vertex(self, vertex):
        if vertex not in self.adjacency_list:
            self.adjacency_list[vertex] = {'name': vertex, 'incoming_edges': 0}

    def add_edge(self, source, target, weight=1):
        if source in self.adjacency_list and target in self.adjacency_list:
            if target in self.adjacency_list[source]:
                # Edge already exists, increase the weight
                self.adjacency_list[source][target] += weight
            else:
                # Edge doesn't exist, add a new edge
                self.adjacency_list[source][target] = weight
                self.adjacency_list[target]['incoming_edges'] += 1

    def display_adjacency_list(self):
        adjacency_list_df = pd.DataFrame(columns=['Zone','weight'])
        for vertex, data in self.adjacency_list.items():
            for neighbor, weight in self.adjacency_list[vertex].items():
                if neighbor != 'name' and neighbor != 'incoming_edges':
                    neighbor_name = self.adjacency_list[neighbor]['name']
                    print(f"{neighbor_name}, {weight} times")
            print(f"{data['name']} has {data['incoming_edges']} incoming rides.")
            print()
            
    def display_adjacency_list_for_vertex(self, vertex_name):
        if vertex_name in self.adjacency_list:
            data = self.adjacency_list[vertex_name]
            adjacency_list_df = pd.DataFrame(columns=['Zone','Weight'])
            for neighbor, weight in data.items():
                if (neighbor != 'name' and neighbor != 'incoming_edges'):
                    if ((self.adjacency_list[neighbor]['name'] != 264) and (self.adjacency_list[neighbor]['name'] != 265)):
                        neighbor_name = self.adjacency_list[neighbor]['name']
                        adjacency_list_df.loc[len(adjacency_list_df.index)] = [self.adjacency_list[neighbor]['name'], weight]
            adjacency_list_df = adjacency_list_df.sort_values(by='Weight', ascending = False)
            return(adjacency_list_df)
        else:
            print(f"Vertex with name {vertex_name} not found in the graph.")
            return(adjacency_list_df)

In [6]:
my_graph = Graph()
# get the current time in seconds
start = time.time()

for i in zone_lookup_df['LocationID']:
    my_graph.add_vertex(i)

for i in range(len(yellow_taxi_df)):
    my_graph.add_edge(yellow_taxi_df['DOLocationID'][i], yellow_taxi_df['PULocationID'][i], 1)

end = time.time()
print("Time to Create Graph:", end-start)

Time to Create Graph: 1.4524142742156982


In [ ]:
def scale_and_shift_geometry(geometry, scale_factor, shift_x, shift_y):
    if geometry.geom_type == 'Polygon':
        return scale_and_shift_polygon(geometry, scale_factor, shift_x, shift_y)
    elif geometry.geom_type == 'MultiPolygon':
        polygons = [scale_and_shift_polygon(p, scale_factor, shift_x, shift_y) for p in geometry.geoms]
        return MultiPolygon(polygons)
    else:
        raise ValueError("Unsupported geometry type")

def scale_and_shift_polygon(polygon, scale_factor, shift_x, shift_y):
    # Extract coordinates from the polygon
    coords = list(polygon.exterior.coords)

    # Scale and shift the coordinates, and flip the y-coordinates
    scaled_and_shifted_coords = [
        (int((x * scale_factor) + shift_x), int((900 - (y) * scale_factor) + shift_y))
        for x, y in coords
    ]

    # Return the scaled and shifted polygon
    return Polygon(scaled_and_shifted_coords)

def draw_scaled_and_shifted_geometry(screen, geometry, color):
    if geometry.geom_type == 'Polygon':
        draw_scaled_and_shifted_polygon(screen, geometry, color)
    elif geometry.geom_type == 'MultiPolygon':
        for polygon in geometry.geoms:
            draw_scaled_and_shifted_polygon(screen, polygon, color)
            

def draw_scaled_and_shifted_polygon(screen, polygon, color):
    exterior_coords = [(int(x), int(y)) for x, y in polygon.exterior.coords]
    pygame.draw.polygon(screen, color, exterior_coords)
    pygame.draw.polygon(screen, (0, 0, 0), exterior_coords, 1)

# Create a pygame window
pygame.init()
screen = pygame.display.set_mode((1200, 1000))
pygame.display.set_caption("Scaled and Shifted Shapefile Visualization")

# Define the scale factor and shift values (adjust as needed)
scale_factor = 0.004
shift_x = -800000 * scale_factor
shift_y = 550
# Clear the screen
screen.fill((0, 0, 0))

# Draw all geometries in the GeoDataFrame
def drawpolygons(color):
    for geometry in gdf['geometry']:
        scaled_and_shifted_geometry = scale_and_shift_geometry(geometry, scale_factor, shift_x, shift_y)
        draw_scaled_and_shifted_geometry(screen, scaled_and_shifted_geometry, color)

base_color = (0,185,140)                        
drawpolygons(base_color)          
                                         
# Main game loop
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if event.type == pygame.MOUSEBUTTONUP:
            screen.fill((0, 0, 0))
            drawpolygons(base_color) 
            i = 0
            for geometry in gdf['geometry']:
                i += 1
                scaled_and_shifted_polygon = scale_and_shift_geometry(geometry, scale_factor, shift_x, shift_y)
                if (scaled_and_shifted_polygon.contains(Point(pygame.mouse.get_pos()))):
                    print(i)
                    adjacency_df = my_graph.display_adjacency_list_for_vertex(i)
                    clicked_color = (200,0,170) 
                    draw_scaled_and_shifted_geometry(screen, scaled_and_shifted_polygon, clicked_color)
                    for j in range(len(adjacency_df.head())):
                        poly = gdf['geometry'].iloc[adjacency_df['Zone'].iloc[j]-1]
                        scaled_and_shifted_polygon = scale_and_shift_geometry(poly, scale_factor, shift_x, shift_y)
                        adjacent_color = (int((255/adjacency_df['Weight'].max())*adjacency_df['Weight'].iloc[j]),0,100) 
                        draw_scaled_and_shifted_geometry(screen, scaled_and_shifted_polygon, adjacent_color)

    # Update the display
    pygame.display.flip()

# Quit pygame
pygame.quit()


107
129
37
26


In [24]:
my_graph = Graph()
# get the current time in seconds
start = time.time()

for i in zone_lookup_df['LocationID']:
    my_graph.add_vertex(i)

for i in range(len(yellow_taxi_df)):
    my_graph.add_edge(yellow_taxi_df['DOLocationID'][i], yellow_taxi_df['PULocationID'][i], 1)

end = time.time()
print("Time to Create Graph:", end-start)

df = my_graph.display_adjacency_list_for_vertex(95)

print(df.head())
print(len(gdf))
print(df['Zone'].iloc[2]-2)
poly = gdf['geometry'].iloc[df['Zone'].iloc[2]-2]

Time to Create Graph: 1.3917813301086426
    Zone  Weight
1    132     138
0    138      53
34    70       7
9     95       7
7    141       4
263
68
